In [ ]:
#!pip install msgraph-sdk==1.0.0a16

In [ ]:
#!pip install msgraph-core==1.0.0a4

In [4]:
from dotenv import load_dotenv 
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
import os
import asyncio 

In [5]:
load_dotenv()

True

In [6]:
TENANT_ID = os.getenv("TENANT_ID")

In [7]:
CLIENT_ID = os.getenv("CLIENT_ID")

In [8]:
graphUserScopes = os.getenv("graphUserScopes").split(' ')

In [9]:
print(CLIENT_ID)

48943d42-a075-43c5-8d93-3f2da879c1be


In [10]:
fabs = 'fabian@fabster.onmicrosoft.com'

In [11]:
adele = 'adelev@fabster.onmicrosoft.com'

In [12]:
## ensure that you have turned on DefaultClientType to Yes in the App Registration

In [13]:
print(graphUserScopes)

['User.Read', 'Mail.Read', 'Mail.Send']


In [14]:
credential = DeviceCodeCredential(client_id=CLIENT_ID, tenant_id=TENANT_ID) 
scopes = graphUserScopes
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [16]:
result = await client.me.get()

In [17]:
print(result.display_name)

Adele Vance


In [18]:
async def get_user(currUser):
    result = await client.users.by_user_id(currUser).get()
    if result:
        print(result.display_name)

In [19]:
await (get_user('adelev@fabster.onmicrosoft.com'))

Adele Vance


In [29]:
async def get_user_messages(currUser):
    messages = await (client.users.by_user_id(currUser).messages.get())
    if messages and messages.value:
        for msg in messages.value:
            print(msg.subject, msg.id, msg.from_, msg.body_preview)

In [30]:
await get_user_messages(adele)

Your digest email AQMkADQ1ZjQzNTE5LTRjNGItNDI1ZC1hNWY1LWNmZTEzNjBjMGY3MgBGAAADTsefQxogGUq-s09IsB8ApgcAR7eBlyFi6U2kmUMoQtaH_AAAAgEMAAAAR7eBlyFi6U2kmUMoQtaH_AACjhcA_wAAAA== Recipient(additional_data={}, email_address=EmailAddress(additional_data={}, address='viva-noreply@microsoft.com', name='Microsoft Viva', odata_type=None), odata_type=None) Private to you



Hi, Adele Vance,
Discover trends in your work habits
An in-depth look at your work patterns in the last four weeks


Explore your insights


27 days without quiet hours interruptions
Days with no significant collaboration ou
Your digest email AQMkADQ1ZjQzNTE5LTRjNGItNDI1ZC1hNWY1LWNmZTEzNjBjMGY3MgBGAAADTsefQxogGUq-s09IsB8ApgcAR7eBlyFi6U2kmUMoQtaH_AAAAgEMAAAAR7eBlyFi6U2kmUMoQtaH_AAChJourAAAAA== Recipient(additional_data={}, email_address=EmailAddress(additional_data={}, address='viva-noreply@microsoft.com', name='Microsoft Viva', odata_type=None), odata_type=None) Private to you



Hi, Adele Vance,
Discover trends in your work habit